In [54]:
import cv2
import numpy as np
from skimage.morphology import closing, disk
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.draw import polygon
from skimage.feature import canny
from scipy.ndimage.morphology import distance_transform_edt
from skimage.transform import resize
import pandas as pd
from utils.process_images import *
from utils.process_data import *

C:\Users\Taqana\AppData\Local\Temp\ipykernel_11380\3012124106.py:8: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import distance_transform_edt


In [55]:
def process_single_image(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    thresh_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    clean_img = closing(thresh_img > 0, disk(1))

    # Calculate stroke thickness
    thickness = stroke_thickness(clean_img)
    mean_thickness = np.mean(thickness)
    std_thickness = np.std(thickness)

    # Calculate number of pixels, end points, and intersection points
    num_pixels = np.sum(clean_img)
    num_ep = number_of_end_points(clean_img, k_nn)
    num_inters = number_of_intersection_points(clean_img, k_nn)

    # Calculate interactions between features
    interactions = {
        'mean_thickness_std_thickness': mean_thickness * std_thickness,
        'mean_thickness_num_pixels': mean_thickness * num_pixels,
        'mean_thickness_num_ep': mean_thickness * num_ep,
        'mean_thickness_num_inters': mean_thickness * num_inters,
        'std_thickness_num_pixels': std_thickness * num_pixels,
        'std_thickness_num_ep': std_thickness * num_ep,
        'std_thickness_num_inters': std_thickness * num_inters,
        'num_pixels_num_ep': num_pixels * num_ep,
        'num_pixels_num_inters': num_pixels * num_inters,
        'num_ep_num_inters': num_ep * num_inters
    }

    # Create a DataFrame with the processed features
    feature_data = {
        'mean_thickness': mean_thickness,
        'std_thickness': std_thickness,
        'num_pixels': num_pixels,
        'num_ep': num_ep,
        'num_inters': num_inters,
        **interactions  # Include interactions in the DataFrame
    }

    df = pd.DataFrame([feature_data])

    return df



In [56]:
image_path = 'new_test\healthy\V01HO01.png'  # Replace with the path to your single image
processed_df = process_single_image(image_path)
processed_df

,mean_thickness,std_thickness,num_pixels,num_ep,num_inters,mean_thickness_std_thickness,mean_thickness_num_pixels,mean_thickness_num_ep,mean_thickness_num_inters,std_thickness_num_pixels,std_thickness_num_ep,std_thickness_num_inters,num_pixels_num_ep,num_pixels_num_inters,num_ep_num_inters
0,22.138837,10.930295,116269,10,14,241.984008,2.574060e+06,221.388368,309.943715,1.270854e+06,109.302946,153.024125,1162690,1627766,140


In [45]:
import pickle

# Load the RF model
with open('rf_model.pickle', 'rb') as model_file:
    loaded_rf_model = pickle.load(model_file)



In [57]:
# Make predictions using the loaded model
predictions = loaded_rf_model.predict(processed_df )

# The 'predictions' variable now contains the predicted labels for each row in processed_df
print(predictions)


[1]
